# Movie Data Analysis

## Introduction
Microsoft Studios is a new comer to the already well established film business. They have everything they need to get started, but they lack a sense of direction and vision. This is where I come in and give them some suggestions from data I have acquired, cleaned, and ran to find trends that can guarantee Microsoft Studios a profitable start. I believe Microsoft Studios should aim to go big on their first film and focus on popularity to build an audience. To find the best way for Microsoft Studios to break into the movie industry, I organized my data around the goal of finding the genre with the highest average rating, highest number of votes, and highest average worldwide grossing. 

## Imports and Collecting Data Frames

In [1]:
import pandas as pd

import sqlite3 

import matplotlib.pyplot as plt

import numpy as np

from matplotlib.ticker import StrMethodFormatter 

def clean_money_cols(x):
    '''
    Quick function to clean money columns to turn the string into a float
    '''
    return float(x.replace('$','').replace(',',''))

In [2]:
#gets rid of scientific notation 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
!ls

Microsoft Studios Notebook.ipynb rt.reviews.tsv.gz
bom.movie_gross.csv.gz           tmdb.movies.csv.gz
im.db                            tn.movie_budgets.csv.gz
rt.movie_info.tsv.gz


In [4]:
!ls ../zippedData

Microsoft Studios Notebook.ipynb rt.reviews.tsv.gz
bom.movie_gross.csv.gz           tmdb.movies.csv.gz
im.db                            tn.movie_budgets.csv.gz
rt.movie_info.tsv.gz


In [5]:
conn = sqlite3.connect("../zippedData/im.db")
conn

In [6]:
imdb_basics = pd.read_sql('SELECT * FROM movie_basics', conn) 

In [7]:
imdb_principals = pd.read_sql('SELECT * FROM principals', conn)

In [8]:
bom = pd.read_csv('bom.movie_gross.csv.gz') 

In [9]:
#dropped any values that appeared as nan
bom.dropna()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.00,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.00,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.00,664300000,2010
3,Inception,WB,292600000.00,535700000,2010
4,Shrek Forever After,P/DW,238700000.00,513900000,2010
...,...,...,...,...,...
3275,I Still See You,LGF,1400.00,1500000,2018
3286,The Catcher Was a Spy,IFC,725000.00,229000,2018
3309,Time Freak,Grindstone,10000.00,256000,2018
3342,Reign of Judges: Title of Liberty - Concept Short,Darin Southa,93200.00,5200,2018


I dropped null values because it is safe to assume that movies with null values under foreign or domestic gross did not make the cut for the type of movies I wanted in my data frame. 

In [10]:
tmdb_movies = pd.read_csv('tmdb.movies.csv.gz') 
tmdb_movies

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.53,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.70,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.73,2010-03-26,How to Train Your Dragon,7.70,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.52,2010-05-07,Iron Man 2,6.80,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.00,1995-11-22,Toy Story,7.90,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.92,2010-07-16,Inception,8.30,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.60,2018-10-13,Laboratory Conditions,0.00,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.60,2018-05-01,_EXHIBIT_84xxx_,0.00,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.60,2018-10-01,The Last One,0.00,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.60,2018-06-22,Trailer Made,0.00,1


In [11]:
movie_budgets = pd.read_csv('tn.movie_budgets.csv.gz') 

In [12]:
imdb_persons = pd.read_sql('SELECT * FROM persons', conn) 

In [13]:
q1 = pd.read_sql('''
SELECT *
FROM principals AS pr
    JOIN persons AS pe
    ON pr.person_id = pe.person_id
''', conn) 

In [14]:
imdb_ratings = pd.read_sql('SELECT * FROM movie_ratings', conn) 

In [ ]:
#https://stackoverflow.com/a/40435354 resource that fixed duplicate columns 
#Selects movie basics and joins it with movie_ratings but it produces duplicate movie_id columns
q2 = pd.read_sql('''
SELECT *
FROM movie_basics AS ba
    JOIN movie_ratings AS ra
    ON ba.movie_id = ra.movie_id
''', conn) 
#This looks for duplicate columns and flips the boolean value to keep unique columns only
q2 = q2.loc[:,~q2.columns.duplicated()] 

In [ ]:
query = '''
SELECT * 
FROM movie_ratings 
ORDER BY averagerating DESC
''' 
q3 = pd.read_sql(query, conn)

In [ ]:
q2.merge(q3,how='left',on=['movie_id'])

I began merging data frames together so that towards the end I could pull all the data from one place.

In [ ]:
q2 = q2.sort_values('averagerating', ascending=False)

In [ ]:
top_votes = q2.sort_values('numvotes', ascending=False)

I sorted the values from greatest to least so I could see the movies with high ratings and high votes first since that's my main focus. 

In [ ]:
#https://www.geeksforgeeks.org/merge-two-pandas-dataframes-on-certain-columns/ 
movie_budgets['release_date'] = movie_budgets['release_date'].astype('datetime64[ns]').dt.year

In [ ]:
for column in ['production_budget', 'domestic_gross', 'worldwide_gross']:
    movie_budgets[column] = movie_budgets[column].apply(clean_money_cols)

In [ ]:
movie_budgets = movie_budgets[movie_budgets['domestic_gross'] > 100000000] 

I narrowed down my data frame to only include movies that grossed over 100 million domestically. 

## Creating the Data Frame

In [ ]:
# I created this variable to merge the data frames together to make one large data frame
top_grossvotes = pd.merge(top_votes, movie_budgets, 
                          left_on=['primary_title','start_year'], 
                          right_on=['movie', 'release_date']) 

In [ ]:
imdb_directors = pd.merge(imdb_principals, imdb_persons,
                          left_on=['person_id'],
                          right_on= ['person_id']) 

In [ ]:
imdb_directors = imdb_directors[imdb_directors['category']=='director'] 

In [ ]:
final_df = top_grossvotes.merge(imdb_directors, left_on='movie_id', right_on='movie_id', how='left') 

In [ ]:
final_df = final_df.drop_duplicates(subset=['primary_title']).reset_index()

In [ ]:
final_df = final_df.drop(columns=['movie_id',
                       'birth_year', 'death_year',
                      'job', 'person_id', 
                      'primary_profession',
                      'ordering', 'index', 'characters', 
                      'runtime_minutes', 'primary_title', 
                      'movie']) 
final_df

## Creating the Visuals

I wanted to focus on finding the most popular genres through highest number of votes, ratings, and worldwide grossing. I also made sure that the genre appeared at least 3 times to be more accurate. If I did not create a minimum count of 3, the data would have been skewed and Drama, Western would have been the most popular despite Django Unchained being the only Drama, Western in the entire data frame!

### Average Votes per Genre

I wanted to present the average number of votes per genre because more votes means more popularity for the film. 

In [ ]:
votes_per_genre = final_df.groupby('genres')['numvotes'].agg(['mean', 'count']) 

In [ ]:
votes_per_genre = votes_per_genre.loc[votes_per_genre['count'] >= 3] 

In [ ]:
top_votes_per_genre = votes_per_genre.sort_values(by='mean').tail(10)

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))
ax.barh(y=top_votes_per_genre.index, width=top_votes_per_genre['mean'])
#https://stackoverflow.com/questions/28371674/prevent-scientific-notation-in-matplotlib-pyplot 
#helped get rid of scientific notation 

ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
#ax.ticklabel_format(useOffset=False, style='plain')
# x = final_df.index
# y = final_df.values 

ax.set_xlabel('Average Number of Votes')
ax.set_ylabel('Types of Genres') 
ax.set_title('Popular Genres')

plt.savefig('../images/numvotes.jpg', bbox_inches='tight', pad_inches=.25)

This chart shows the genres with the highest average number of votes.

### Average Rating per Genre

I wanted to find the average rating per genre because it is important that Microsoft Studios makes a good first impression in the film industry.

In [ ]:
top_ratings_per_genre = final_df.groupby('genres')['averagerating'].agg(['mean', 'count']) 

In [ ]:
top_ratings_per_genre = top_ratings_per_genre.loc[top_ratings_per_genre['count'] >= 3] 

In [ ]:
top_ratings_per_genre = top_ratings_per_genre.sort_values(by='mean').tail(20)

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))

colorlist = np.where(top_ratings_per_genre['mean'] > 6.9, 'green', 'blue')

ax.barh(y=top_ratings_per_genre.index, width=top_ratings_per_genre['mean'], color=colorlist)

ax.set_xlabel('Average Rating out of 10')
ax.set_ylabel('Types of Genres') 
ax.set_title('Highly Rated Genres')
# ax.set_xlim(6,8)

plt.savefig('../images/ratedgenrescolorcoded.jpg', bbox_inches='tight', pad_inches=.25)

This chart shows the highest average rating for each genre. 

### Average Worldwide Grossing per Genre

I wanted to find the average worldwide grossing to give stakeholders an estimate on how much money they could make with the genre they selected.

In [ ]:
average_worldwide_gross = final_df.groupby('genres')['worldwide_gross'].agg(['mean', 'count']) 

In [ ]:
average_worldwide_gross = average_worldwide_gross.loc[average_worldwide_gross['count'] >= 3] 

In [ ]:
average_worldwide_gross = average_worldwide_gross.sort_values(by='mean').tail(10)

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))
ax.barh(y=average_worldwide_gross.index, width=average_worldwide_gross['mean'])

ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

ax.set_xlabel('Average Worldwide Gross')
ax.set_ylabel('Types of Genres') 
ax.set_title('Worldwide Grossing') 

plt.savefig('../images/worldwidegross.jpg', bbox_inches='tight', pad_inches=.25)

This chart shows the average worldwide grossing for each genre.

### Average Production Budget per Genre

I wanted to find the average production budget to give stakeholders an estimate on how much they would be spending.

In [ ]:
average_budget = final_df.groupby('genres')['production_budget'].agg(['mean', 'count'])

In [ ]:
average_budget = average_budget.loc[average_budget['count'] >= 3] 

In [ ]:
average_budget = average_budget.sort_values(by='mean').tail(10) 

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))
ax.barh(y=average_budget.index, width=average_budget['mean'])

ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

ax.set_xlabel('Average Budget')
ax.set_ylabel('Types of Genres') 
ax.set_title('Average Production Budget Across Genres')

plt.savefig('../images/averagebudget.jpg', bbox_inches='tight', pad_inches=.25)

This chart shows the average production budget for each genre. 

## Conclusion 
The findings from my research showed that the most popular movie genre, which I measured by average number of votes and average rating, is the combination of Action and Adventure. There were many different sub-genres of Action and Adventure such as Thriller, Sci-fi, Animation, Comedy, etc. I recommended Microsoft Studios to make an Action Adevnture film with a thriller aspect because they typically gross the highest compared to the other types of Action Adventure films and are towards the bottom in average production budget.